**CTE: Common Table Expression**

**RECURSIVE CTE**

**مثال:**

 تولید اعداد 1 تا 100

In [ ]:
WITH Numbers
AS
(
	-- Anchor Part
	SELECT 1 AS N

	UNION ALL
	-- Recursion Part
	SELECT 
		N + 1
	FROM
		Numbers
	WHERE
		N < 10
)

SELECT 
	*
FROM
	Numbers

ساخت جدول پست های سازمانی

In [ ]:
USE SQLCLass
GO

CREATE TABLE dbo.Post
(
	ID INT PRIMARY KEY ,
	ParentID INT REFERENCES dbo.Post(ID),
	Title	NVARCHAR(50)
)
GO

INSERT INTO dbo.Post
VALUES
	(1, NULL, N'مدیر عامل'),
	(2, 1, N'مدیر مالی'),
	(3, 1, N'مدیر فروش'),
	(4, 1, N'مدیر منابع انسانی'),
	(5, 2, N'سرپرست حسابداری'),
	(6, 5, N'حسابدار 1'),
	(7, 5, N'حسابدار 2'),
	(8, 3, N'سرپرست مناطق'),
	(9, 8, N'مدیر منطقه شمال'),
	(10, 8, N'مدیر منطقه جنوب'),
	(11, 4, N'رییس کارگزینی'),
	(12, 11, N'کارشناس کارگزینی'),
	(13, 11, N'کارشناس حقوق و دستمزد'),
	(14, 11, N'کارشناس تردد')


In [ ]:
SELECT
	*
FROM
	dbo.Post

**مثال**:

پیمایش پست ها از بالاترین شاخه به پایین

In [ ]:
WITH CTE
AS
(
	SELECT
		*,
		1 AS Level
	FROM
		dbo.Post
	WHERE
		ParentID IS NULL

	UNION ALL
	SELECT 
		C.*,
		P.Level + 1
	FROM
		CTE AS P
	INNER JOIN
		dbo.Post AS C
	ON
		P.ID = C.ParentID
)

SELECT 
	*
FROM
	CTE

**مثال:**  
  
نوشتن یک فانکشن که پست های زیر مجموعه یک پست را پیمایش کند

In [ ]:
CREATE OR ALTER FUNCTION dbo.tvfPost
(
	@PostID INT
)
RETURNS TABLE
AS 
RETURN
(
	WITH CTE
	AS
	(
		SELECT
			*,
			1 AS Level
		FROM
			dbo.Post
		WHERE
			ID = @PostID

		UNION ALL
		SELECT 
			C.*,
			P.Level + 1
		FROM
			CTE AS P
		INNER JOIN
			dbo.Post AS C
		ON
			P.ID = C.ParentID
	)

	SELECT 
		*
	FROM
		CTE
)
GO
SELECT 
	*
FROM
	dbo.tvfPost(2)

مثال: 

پیمایش پست ها با کمی زیبا سازی

In [ ]:
WITH CTE
AS
(
	SELECT
		*,
		1 AS Level,
		CONVERT(VARCHAR(MAX), Title) AS Path1,
		CONVERT(VARCHAR(MAX), Title) AS Path2
	FROM
		dbo.Post
	WHERE
		ParentID IS NULL

	UNION ALL
	SELECT 
		C.*,
		P.Level + 1,
		CONVERT(VARCHAR(MAX), P.Path1 + ' --> ' + C.Title) AS Path1,
		CONVERT(VARCHAR(MAX), REPLICATE('				|				', P.level) + C.Title) AS Path2
	FROM
		CTE AS P
	INNER JOIN
		dbo.Post AS C
	ON
		P.ID = C.ParentID
)

SELECT 
	*
FROM
	CTE